In [2]:
import parselmouth
from parselmouth.praat import call
import pandas as pd
import csv
import numpy as np
from multiprocessing import  Pool
import time

### 1.1

In [3]:
def get_data(file):
    df = pd.read_csv(file)
    return df 

In [17]:
train = get_data("train_merged_timestamps_liwc_fixed_iteration1.csv")
test = get_data("test_merged_timestamps_liwc_fixed.csv")

/Users/tejitpabari/Desktop/ML/ML_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
train.head()

,Unnamed: 0,swda_filename,ptb_basename,conversation_no,transcript_index,act_tag,caller,utterance_index,subutterance_index,original_text,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0,sw00utt/sw_0002_4330.utt,4/sw4330,4330,0,qy,A,1,1,"{D Well, } {F uh, } does the company you work ...",...,0,0.0,0.0,0,0.0,0.00,0,0.00,0,0.0
1,1,sw00utt/sw_0002_4330.utt,4/sw4330,4330,1,na,B,2,1,"{D Actually, } {F uh, } they just recently sta...",...,0,0.0,0.0,0,0.0,0.00,0,0.00,0,0.0
2,2,sw00utt/sw_0002_4330.utt,4/sw4330,4330,2,sd^e,B,2,2,"{C because } [ w-, + when ] I went to work for...",...,0,0.0,0.0,0,0.0,7.14,0,7.14,0,0.0
3,3,sw00utt/sw_0002_4330.utt,4/sw4330,4330,3,sd,B,2,3,"{C but, } {F uh, } since then [ they've, + the...",...,0,0.0,0.0,0,0.0,0.00,0,6.45,0,0.0
4,4,sw00utt/sw_0002_4330.utt,4/sw4330,4330,4,%,A,3,1,{F Huh. } /,...,0,0.0,0.0,0,0.0,0.00,0,0.00,0,0.0


In [31]:
train = train.loc[ train['act_tag'].isin( ['sd', 'b', 'sv', 'aa', '%', 'ba', 'qy', 'qw', 'ny', 'fc'])] 
train = train.loc[ train['end_time']-train['start_time'] > 0] 
train.count()

Unnamed: 0          131575
swda_filename       131575
ptb_basename        131575
conversation_no     131575
transcript_index    131575
                     ...  
Dash                131575
Quote               131575
Apostro             131575
Parenth             131575
OtherP              131575
Length: 102, dtype: int64

#### Choosing Speech Based Features - Using Top 1000

In [64]:
def get_pitch(sound, start, end):

    pitch_obj = call(sound, "To Pitch", 0.0, 75, 600)  # create a praat pitch object
    mean_pitch = call(pitch_obj, "Get mean", start, end, 'Hertz')  # get mean pitch
    max_pitch = call(pitch_obj, "Get maximum", start, end, 'Hertz', "None")
    min_pitch = call(pitch_obj, "Get minimum", start, end, 'Hertz', "None")
    diff, increase = max_pitch - min_pitch, mean_pitch - min_pitch
    if diff == 0: normalized_mean_pitch = 0
    else: normalized_mean_pitch = 2*increase/diff - 1
    sd_pitch = call(pitch_obj, "Get standard deviation", start, end, 'Hertz')

    return (normalized_mean_pitch, sd_pitch)

def get_intensity(sound, start, end):

    intensity_obj = call(sound, "To Intensity", 75, 0, "yes")
    mean_intensity = call(intensity_obj, "Get mean", start, end, 'dB')
    max_intensity = call(intensity_obj, "Get maximum", start, end, "None")
    min_intensity = call(intensity_obj, "Get minimum", start, end, "None")
    diff, increase = max_intensity - min_intensity, mean_intensity - min_intensity
    if diff == 0: normalized_mean_intensity = 0
    else: normalized_mean_intensity = 2*increase/diff - 1
    sd_intensity = call(intensity_obj, "Get standard deviation", start, end)

    return (normalized_mean_intensity, sd_intensity)

def extract_features(recording, words, start, end):
    sound = parselmouth.Sound(recording)
    mean_pitch, sd_pitch = get_pitch(sound, start, end)
    mean_intensity, sd_intensity = get_intensity(sound, start, end)

    duration = end-start
    speaking_rate = words/duration
    
    PointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 600)
    jitter = call(PointProcess, "Get jitter (local)", start, end, 0.0001, 0.02, 1.3)
    shimmer = call([sound, PointProcess], "Get shimmer (local)", start, end, 0.0001, 0.02, 1.3, 1.6)

    Harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1)
    hnr = call(Harmonicity, "Get mean", start, end)
    
    return (mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate, jitter, shimmer, hnr)

In [65]:
train = train[['WC', 'start_time', 'end_time', 'conversation_no', 'act_tag']]
# train = train.head(1000)

In [72]:
train2 = train[train['conversation_no'].isin(range(2005, 2006))].sort_values('conversation_no')
print(len(train2))
print(sorted(set(train2['conversation_no'])))
train2.head()

193
[2005]


,WC,start_time,end_time,conversation_no,act_tag
169894,42,10.99,21.19,2005,sv
170044,28,331.28,337.96,2005,sd
170045,1,337.96,338.36,2005,b
170047,1,340.76,341.00,2005,b
170049,6,341.32,342.80,2005,sd


In [67]:
def speech_append(l):
    words = l['WC']
    start = l['start_time']
    end = l['end_time']
    if end-start > 0:
        file = "wav_files/sw" + str(l['conversation_no']) + ".wav"
        tag = l['act_tag']

        mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate, jitter, shimmer, hnr = extract_features(file, 
                                                                                                       words, 
                                                                                                       start, 
                                                                                                       end)
        return mean_pitch, mean_intensity, sd_pitch, sd_intensity, duration, speaking_rate, jitter, shimmer, hnr, tag

In [68]:
def apply(df2):
    df2[ 'mean_pitch'], df2['mean_intensity'],df2['sd_pitch'], df2['sd_intensity'], df2['duration'], df2['speaking_rate'], df2['jitter'], df2['shimmer'], df2['hnr'], df2['tag'] = zip(*df2.apply(speech_append,axis=1))
    return df2

In [69]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split), copy=False)
    pool.close()
    pool.join()
    return df

In [70]:
i = 0
while i < (train2.shape[0]-100):
    start = time.time()
    train_i = train2[i:i+100]
#     print('1')
    train_processed = parallelize_dataframe(train_i, apply)
#     print('2')
#     train_csv = train_processed[['mean_pitch', 'mean_intensity','sd_pitch', 'sd_intensity', 'duration', 'speaking_rate', 'jitter', 'shimmer', 'hnr', 'tag']]
#     csv_name = str(i)+'train_1000_features.csv'
#     train_csv.to_csv(csv_name)
    end = time.time()
    print(end-start)
    break
    i += 100

(100, 5)


KeyboardInterrupt: 

Process ForkPoolWorker-61:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-68-e333df19f8af>", line 2, in apply
    df2[ 'mean_pitch'], df2['mean_intensity'],df2['sd_pitch'], df2['sd_intensity'], df2['duration'], df2['speaking_rate'], df2['jitter'], df2['shimmer'], df2['hnr'], df2['tag'] = zip(*df2.apply(speech_append,axis=1))
  File "/Users/tejitpabari/Desktop/ML/ML_venv/lib/pyth

In [103]:
# train_processed[['mean_pitch', 'mean_intensity','sd_pitch', 'sd_intensity', 'duration', 'speaking_rate', 'jitter', 'shimmer', 'hnr', 'tag']].head()

,mean_pitch,mean_intensity,sd_pitch,sd_intensity,duration,speaking_rate,jitter,shimmer,hnr,tag
0,0.439332,-0.233840,23.434147,12.741367,3.92,2.806122,0.016667,0.073849,13.752097,qy
3,-0.333770,-0.065550,15.809203,13.735634,9.64,3.215768,0.017093,0.081315,14.842716,sd
4,-0.195737,0.023572,3.449232,11.928231,2.20,0.454545,0.009435,0.081687,16.161668,%
8,-0.238647,-0.056787,18.148676,12.747733,5.92,3.378378,0.007669,0.071836,16.237164,sd
9,0.093581,0.431376,5.741084,4.398988,0.12,8.333333,0.008277,0.097482,NaN,b
11,0.288868,0.119058,35.080623,10.062030,3.32,2.409639,0.013911,0.085994,9.021464,sd
12,0.338253,-0.272923,41.637152,10.579021,2.48,0.403226,0.017110,0.100653,NaN,%
13,-0.566523,0.292485,80.277510,11.680172,1.48,2.702703,0.013393,0.089717,14.755235,sv
14,0.144820,0.240276,19.578033,10.058456,2.64,3.409091,0.011318,0.074671,16.280208,sd
15,-0.502173,0.182481,36.441011,9.895164,0.72,1.388889,0.024810,0.102070,5.681322,b


In [ ]:
# train_csv = train_processed[['mean_pitch', 'mean_intensity','sd_pitch', 'sd_intensity', 'duration', 'speaking_rate', 'jitter', 'shimmer', 'hnr', 'tag']]

In [ ]:
# train_csv.to_csv('train_1000_features.csv')

#### Extract Speech Based Features

In [6]:
def get_pitch(sound, start, end):

    pitch_obj = call(sound, "To Pitch", 0.0, 75, 600)  # create a praat pitch object
    mean_pitch = call(pitch_obj, "Get mean", start, end, 'Hertz')  # get mean pitch
    max_pitch = call(pitch_obj, "Get maximum", start, end, 'Hertz', "None")
    min_pitch = call(pitch_obj, "Get minimum", start, end, 'Hertz', "None")
    diff, increase = max_pitch - min_pitch, mean_pitch - min_pitch
    if diff == 0: normalized_mean_pitch = 0
    else: normalized_mean_pitch = 2*increase/diff - 1
    sd_pitch = call(pitch_obj, "Get standard deviation", start, end, 'Hertz')

    return (normalized_mean_pitch, sd_pitch)

def get_intensity(sound, start, end):

    intensity_obj = call(sound, "To Intensity", 75, 0, "yes")
    mean_intensity = call(intensity_obj, "Get mean", start, end, 'dB')
    max_intensity = call(intensity_obj, "Get maximum", start, end, "None")
    min_intensity = call(intensity_obj, "Get minimum", start, end, "None")
    diff, increase = max_intensity - min_intensity, mean_intensity - min_intensity
    if diff == 0: normalized_mean_intensity = 0
    else: normalized_mean_intensity = 2*increase/diff - 1
    sd_intensity = call(intensity_obj, "Get standard deviation", start, end)

    return (normalized_mean_intensity, sd_intensity)

def extract_features(recording, words, start, end):
    sound = parselmouth.Sound(recording)
    mean_pitch, sd_pitch = get_pitch(sound, start, end)
    mean_intensity, sd_intensity = get_intensity(sound, start, end)

    duration = end-start
    speaking_rate = words/duration

    return (mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate)


In [44]:
train = train[['WC', 'start_time', 'end_time', 'conversation_no', 'act_tag']]
train = train.head(1000)
train

,WC,start_time,end_time,conversation_no,act_tag
0,11,0.00,3.92,4330,qy
3,31,12.28,21.92,4330,sd
4,1,21.92,24.12,4330,%
8,20,28.44,34.36,4330,sd
9,1,34.36,34.48,4330,b
11,8,35.12,38.44,4330,sd
12,1,38.44,40.92,4330,%
13,4,40.92,42.40,4330,sv
14,9,42.40,45.04,4330,sd
15,1,45.04,45.76,4330,b


In [65]:
def speech_append(l):
    words = l['WC']
    start = l['start_time']
    end = l['end_time']
    if end-start > 0:
        file = "wav_files/sw" + str(l['conversation_no']) + ".wav"
        tag = l['act_tag']

        mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate = extract_features(file, 
                                                                                                       words, 
                                                                                                       start, 
                                                                                                       end)
        return mean_pitch, mean_intensity, sd_pitch, sd_intensity, duration, speaking_rate, tag

In [78]:
def apply(df2):
    df2[ 'mean_pitch'], df2['mean_intensity'],df2['sd_pitch'], df2['sd_intensity'], df2['duration'], df2['speaking_rate'], df2['tag'] = zip(*df2.apply(speech_append,axis=1))
    return df2


In [79]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split), copy=False)
    pool.close()
    pool.join()
    return df

In [71]:
train3 = train2.head(1)
start = time.time()
train_processed = parallelize_dataframe(train3, apply)
end = time.time()
print(end-start)

ValueError: not enough values to unpack (expected 10, got 2)

In [84]:
train_processed[['mean_pitch', 'mean_intensity','sd_pitch', 'sd_intensity', 'duration', 'speaking_rate', 'tag']]

,mean_pitch,mean_intensity,sd_pitch,sd_intensity,duration,speaking_rate,tag
0,0.439332,-0.233840,23.434147,12.741367,3.92,2.806122,qy
3,-0.333770,-0.065550,15.809203,13.735634,9.64,3.215768,sd
4,-0.195737,0.023572,3.449232,11.928231,2.20,0.454545,%
8,-0.238647,-0.056787,18.148676,12.747733,5.92,3.378378,sd
9,0.093581,0.431376,5.741084,4.398988,0.12,8.333333,b
11,0.288868,0.119058,35.080623,10.062030,3.32,2.409639,sd
12,0.338253,-0.272923,41.637152,10.579021,2.48,0.403226,%
13,-0.566523,0.292485,80.277510,11.680172,1.48,2.702703,sv
14,0.144820,0.240276,19.578033,10.058456,2.64,3.409091,sd
15,-0.502173,0.182481,36.441011,9.895164,0.72,1.388889,b


In [39]:
# data = []

# def speech_append(l):
#     words = l[0]
#     start = l[1]
#     end = l[2]
#     if end-start > 0:
#         file = "wav_files/sw" + str(l[3]) + ".wav"
#         tag = l[4]

#         mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate = extract_features(file, 
#                                                                                                        words, 
#                                                                                                        start, 
#                                                                                                        end)
#         data.append([mean_pitch, mean_intensity, sd_pitch, sd_intensity, duration, speaking_rate, tag]) 
# train.apply(lambda row: speech_append(row),axis=1)
    
# data = []
# for index, row in train.iterrows():
#     words = row['WC']
#     start = row['start_time']
#     end = row['end_time']
#     if end-start > 0:
#         file = "wav_files/sw" + str(row['conversation_no']) + ".wav"
#         tag = row['act_tag']

#         mean_pitch, sd_pitch, mean_intensity, sd_intensity, duration, speaking_rate = extract_features(file, 
#                                                                                                        words, 
#                                                                                                        start, 
#                                                                                                        end)
#         data.append([mean_pitch, mean_intensity, sd_pitch, sd_intensity, duration, speaking_rate, tag]) 
# train_data = pd.DataFrame(data, columns = ['normalized pitch', 'normalized intensity', 'sd pitch', 
#                                            'sd intensity', 'duration', 'speaking rate', 'tag'])
# train_data.count()

#### Extract Text Based Features 

In [ ]:
# Exploring the data relationships
top10 = []
for 